In [ ]:
'''
To convert a text input into an lstm type input-output
'''
import numpy as np
from keras.utils.np_utils import to_categorical

Using Theano backend.


In [2]:
SAMPLE = '''abcdefghijklmnopqrstuvwxyz'''*10

In [3]:
SEQ_LEN = 12
BATCH_SIZE = 10
VOCAB_SIZE = len(list(set(SAMPLE)))
EMBED_SIZE = 13
LSTM_LAYERS = 3

In [4]:
VOCAB_SIZE, EMBED_SIZE

(26, 13)

In [5]:
chars = sorted(list(set(SAMPLE)))
char_to_idx = {char:i for i, char in enumerate(chars)}
idx_to_char = {i:char for char, i in char_to_idx.items()}

In [6]:
char_to_idx

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [7]:
starts = np.random.randint(0, high = 260 - SEQ_LEN - 1, size = 2)
starts
def create_batch(batch_size = BATCH_SIZE, sample = SAMPLE):
    starts = np.random.randint(0, high = len(sample) - SEQ_LEN - 1, size = batch_size)
    starts
    x = np.array([[char_to_idx[char] for char in sample[start:start+SEQ_LEN]] for start in starts])
    y = np.array([[char_to_idx[char] for char in sample[start+1:start+1+SEQ_LEN]] for start in starts])    
    return(x, y)

In [8]:
def create_batch(batch_size = BATCH_SIZE, sample = SAMPLE):
    starts = np.random.randint(0, high = len(sample) - SEQ_LEN - 1, size = batch_size)
    starts
    x = np.array([[char_to_idx[char] for char in sample[start:start+SEQ_LEN]] for start in starts])
    y = np.array([to_categorical([char_to_idx[char] for char in sample[start+1:start+1+SEQ_LEN]], nb_classes=VOCAB_SIZE) for start in starts])    
    return(x, y)

In [9]:
x, y = create_batch(2)

x, y 

(array([[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]]),
 array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Dropout
from keras.layers.recurrent import LSTM

In [11]:
x.shape, y.shape

((2L, 12L), (2L, 12L, 26L))

In [12]:
def LSTM_model(infer):
    if infer:
        batch_size = seq_len = 1
    else:
        batch_size = BATCH_SIZE
        seq_len = SEQ_LEN
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, int(EMBED_SIZE), input_length = seq_len, batch_input_shape = (batch_size, seq_len)))
    for _ in range(LSTM_LAYERS):
        model.add(LSTM(8, return_sequences = True, stateful = True))
        model.add(Dropout(0.2))
    model.add(Dense(VOCAB_SIZE))
    model.add(Activation('softmax'))
    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return(model)

In [13]:
train = LSTM_model(False)

In [23]:
from tqdm import trange
iters = trange(1000)
for _ in iters:
    X, y = create_batch()
    train.fit(X, y, batch_size = BATCH_SIZE, nb_epoch=1, verbose=0)
train.evaluate(X, y, batch_size=BATCH_SIZE)




  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


  1%|▋                                                                                | 9/1000 [00:00<00:12, 82.57it/s]


  2%|█▌                                                                              | 19/1000 [00:00<00:11, 83.63it/s]


  3%|██▍                                                                             | 30/1000 [00:00<00:11, 88.16it/s]


  4%|███▎                                                                            | 41/1000 [00:00<00:10, 91.18it/s]


  5%|████                                                                            | 51/1000 [00:00<00:10, 91.35it/s]


  6%|████▉                                                                           | 62/1000 [00:00<00:10, 93.78it/s]


  7%|█████▊                                                                          | 73/1000 [00:00<00:09, 96.07it/s]


  8%|██████▋         

10/10 [==============================] - 0s


[0.23073975741863251, 0.92500001192092896]

In [24]:
X, y = create_batch()
pred = train.predict_classes(X, verbose = 0)

In [25]:
for y in pred:
    print(''.join([idx_to_char[y_] for y_ in y]))

mnopqrstuvwx
ghjklmnopqrs
glmnopqrstuv
tvwxyzabcdef
hijklmnopqrs
vvvwxyzabcde
srstuvwxyzab
zabcdefghijk
cdefghijklmn
ijklmnopqrst


In [45]:
test = LSTM_model(True)
test.set_weights(train.get_weights())

In [47]:
y = []
pred = np.array([[0]])
while len(y) < 200:
    pred = test.predict_classes(pred, verbose = 0)
    y.append(pred[0][0])
    
''.join(idx_to_char[y_] for y_ in y)

'bcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrs'